# Deep Learning Pipeline with Proactive Jupyter Kernel and Tensorboard
The ActiveEon Jupyter Kernel adds a kernel backend to Jupyter.

This kernel interfaces directly with the ProActive scheduler and constructs tasks and workflows to execute them on the fly.

With this interface, users can run their code locally and test it using a native python kernel, and by a simple switch to ProActive kernel, run it on remote public or private infrastructures without having to modify the code.

This notebook was based on the following one:

https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/chapter09_part01_image-segmentation.ipynb

See https://github.com/ow2-proactive/proactive-jupyter-kernel for more information.

As a quick start, we recommend the user to run the `#%help()` pragma using the following script:

In [ ]:
#%help()

## Connection

If you are trying ProActive for the first time, sign up on the [try platform](https://try.activeeon.com/signup.html).

Once you receive your login and password, connect to the trial platform using the `#%connect()` pragma.

For more information, type: `#%help(pragma=connect)`

In [ ]:
#%connect(url=https://try.activeeon.com:8443)

## Runtime environment definition

The `#%runtime_env()` pragma enable user to define the runtime environment for pipeline execution.

The user can select the container type (docker, podman, singularity), the container image, and mount local directories inside container.

For more information, type: `#%help(pragma=runtime_env)`

In [ ]:
#%runtime_env(type=docker,image=activeeon/tensorflow,nvidia_gpu=false,mount_host_path=/shared,mount_container_path=/shared)

, for best performance on NVIDIA GPUs use:

In [ ]:
#%runtime_env(type=docker,image=activeeon/tensorflow:latest-gpu,nvidia_gpu=true,mount_host_path=/shared,mount_container_path=/shared)

## Importing libraries
The main difference between the ProActive and 'native language' kernels resides in the way the memory is accessed
during blocks execution. In a common native language kernel, the whole script code (all the notebook blocks) is
locally executed in the same shared memory space; whereas the ProActive kernel will execute each created task in an
independent process. In order to facilitate the transition from native language to ProActive kernels, we included the
pragma `#%import()`. This pragma gives the user the ability to add libraries that are common to all created tasks, and
thus relative distributed processes, that are implemented in the same native script language.

The import pragma is used as follows:

`#%import([language=SCRIPT_LANGUAGE])`.

Example:

```python
#%import(language=Python)
import os
import pandas
```

NOTE: If the language is not specified, Python is considered as default language.

In [ ]:
#%import()
import os
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

import numpy as np
import random

try:
    from PIL import Image
except:
    install("Pillow==8.4.0")
try:
    import matplotlib.pyplot as plt
except:
    install("matplotlib==3.5.1")
    import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.utils import array_to_img

from pathlib import Path

def path_to_input_image(path):
    return img_to_array(load_img(path, target_size=img_size))

def path_to_target(path):
    img = img_to_array(load_img(path, target_size=img_size, color_mode="grayscale"))
    img = img.astype("uint8") - 1
    return img

## Creating tasks

### Creating the _download_dataset_ task

In [ ]:
#%task(name=download_dataset,language=Linux_Bash)
MODELS_PATH="/shared/models"
mkdir -p $MODELS_PATH

DATASET_PATH="/shared/datasets/pets"
if [ ! -d "$DATASET_PATH" ]; then
    mkdir -p $DATASET_PATH
    apt update && apt install -y wget
    wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz -P $DATASET_PATH
    wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz -P $DATASET_PATH
    tar -xf $DATASET_PATH/images.tar.gz -C $DATASET_PATH
    tar -xf $DATASET_PATH/annotations.tar.gz -C $DATASET_PATH
    echo "Done"
else
    echo "$DATASET_PATH already exists"
fi

### Creating the _train_model_ task 

In [ ]:
#%task(name=train_model,dep=[download_dataset])
DATASET_PATH="/shared/datasets/pets"
input_dir = os.path.join(DATASET_PATH, "images")
target_dir = os.path.join(DATASET_PATH, "annotations/trimaps")

input_img_paths = sorted(
    [os.path.join(input_dir, fname)
     for fname in os.listdir(input_dir)
     if fname.endswith(".jpg")])
target_paths = sorted(
    [os.path.join(target_dir, fname)
     for fname in os.listdir(target_dir)
     if fname.endswith(".png") and not fname.startswith(".")])

img_size = (200, 200)
num_imgs = len(input_img_paths)

random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_paths)

input_imgs = np.zeros((num_imgs,) + img_size + (3,), dtype="float32")
targets = np.zeros((num_imgs,) + img_size + (1,), dtype="uint8")
for i in range(num_imgs):
    input_imgs[i] = path_to_input_image(input_img_paths[i])
    targets[i] = path_to_target(target_paths[i])

num_val_samples = 1000
train_input_imgs = input_imgs[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_input_imgs = input_imgs[-num_val_samples:]
val_targets = targets[-num_val_samples:]

def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))
    x = layers.Rescaling(1./255)(inputs)
    x = layers.Conv2D(64, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(256, 3, strides=2, padding="same", activation="relu")(x)
    x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same", strides=2)(x)
    x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same", strides=2)(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same", strides=2)(x)
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)
    model = keras.Model(inputs, outputs)
    return model

# Create model
model = get_model(img_size=img_size, num_classes=3)
model.summary() # Total params: 2,880,643

# Setup optmizer and loss function
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

# Setup Tensorboard
if 'variables' in globals():
    PA_JOB_ID = variables.get("PA_JOB_ID")
    PA_TRIAL_ID = variables.get("PA_TASK_REPLICATION") # or PA_TASK_ID
    TENSORBOARD_LOG_PATH = "/shared/tensorboard/job_id_" + str(PA_JOB_ID) + "_t" + str(PA_TRIAL_ID)
else:
    TENSORBOARD_LOG_PATH = "./logs/trial"
Path(TENSORBOARD_LOG_PATH).mkdir(parents=True, exist_ok=True)

# Setup callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint("/shared/models/oxford_segmentation.keras", save_best_only=True),
    keras.callbacks.TensorBoard(log_dir=TENSORBOARD_LOG_PATH)
]

# Train model
history = model.fit(train_input_imgs, train_targets,
                    epochs=3,
                    callbacks=callbacks,
                    batch_size=16,
                    validation_data=(val_input_imgs, val_targets))

print("Average training loss: ", np.average(history.history['loss']))
print("Average validation loss: ", np.average(history.history['val_loss']))

# epochs = range(1, len(history.history["loss"]) + 1)
# loss = history.history["loss"]
# val_loss = history.history["val_loss"]

# plt.figure()
# plt.plot(epochs, loss, "bo", label="Training loss")
# plt.plot(epochs, val_loss, "b", label="Validation loss")
# plt.title("Training and validation loss")
# plt.legend()

### Creating the _predict_ task 

In [ ]:
#%task(name=predict,dep=[train_model])
import matplotlib.image as mpimg
import shutil

path = '/shared/datasets/pets/images/chihuahua_1.jpg'
img_size = (200, 200)
test_image = path_to_input_image(path)

model = keras.models.load_model("/shared/models/oxford_segmentation.keras")

pred = model.predict(np.expand_dims(test_image, 0))[0]

mask = np.argmax(pred, axis=-1)
mask *= 127

shutil.copyfile(path, "/shared/input.png")
mpimg.imsave("/shared/prediction.png", mask)

print("Done")

### Visualizing the job pipeline

In [ ]:
#%draw_job()

### Submitting the job to the scheduler

To submit the job to the ProActive Scheduler, the user has to use the `#%submit_job()` pragma:

```python
#%submit_job()
```

If the job is not created, or is not up-to-date, the `#%submit_job()` creates a new job named as the old one.
To provide a new name, use the same pragma and provide a name as parameter:

```python
#%submit_job([name=JOB_NAME])
```

If the job's name is not set, the ProActive kernel uses the current notebook name, if possible, or gives a random one.

In [ ]:
#%submit_job()

### Getting results and outputs

After the execution of a ProActive workflow, two outputs can be obtained,
* results: values that have been saved in the 
[task result variable](https://doc.activeeon.com/latest/user/ProActiveUserGuide.html#_task_result),
* console outputs: classic outputs that have been displayed/printed 

To get task results, please use the `#%get_task_result()` pragma by providing the task name, and either the job ID or
the job name:

```python
#%get_task_result([job_id=JOB_ID], [job_name=JOB_NAME], task_name=TASK_NAME)
```

The result(s) of all the tasks of a job can be obtained with the `#%get_job_result()` pragma, by providing the job name
or the job ID:

```python
#%get_job_result([job_id=JOB_ID], [job_name=JOB_NAME])
```

To get and display console outputs of a task, you can use the `#%print_task_output()` pragma in the following
way:

```python
#%print_task_output([job_id=JOB_ID], [job_name=JOB_NAME], task_name=TASK_NAME)
```

Finally, the  `#%print_job_output()` pragma allows to print all job outputs, by providing the job name or the job ID:

```python
#%print_job_output([job_id=JOB_ID], [job_name=JOB_NAME])
```

NOTE: If neither `job_name` nor the `job_id` are provided, the last submitted job is selected by default. 

In [ ]:
#%print_job_output()